# Gesture

In [18]:
import cv2
import serial
import mediapipe as mp

# Initialize MediaPipe hand model
mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mpDraw = mp.solutions.drawing_utils

# Initialize Serial Port
ser = serial.Serial('/dev/cu.usbserial-140', 9600, timeout=1)

def control_fan(hand_landmarks, hand_type):
    fingers_open = 0

    # Check if each finger is open
    # Adjust thumb detection based on hand type (left or right)
    if hand_type == 'Right':
        if hand_landmarks.landmark[mpHands.HandLandmark.THUMB_TIP].x < hand_landmarks.landmark[mpHands.HandLandmark.THUMB_IP].x:
            fingers_open += 1
    else:  # Left hand
        if hand_landmarks.landmark[mpHands.HandLandmark.THUMB_TIP].x > hand_landmarks.landmark[mpHands.HandLandmark.THUMB_IP].x:
            fingers_open += 1

    # Other fingers
    for finger in [mpHands.HandLandmark.INDEX_FINGER_TIP, mpHands.HandLandmark.MIDDLE_FINGER_TIP, mpHands.HandLandmark.RING_FINGER_TIP, mpHands.HandLandmark.PINKY_TIP]:
        if hand_landmarks.landmark[finger].y < hand_landmarks.landmark[finger - 2].y:
            fingers_open += 1

    ser.write(str(fingers_open).encode())
    return fingers_open

def main():
    cap = cv2.VideoCapture(0)

    try:
        while cap.isOpened():
            ret, img = cap.read()
            if not ret:
                continue

            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(imgRGB)

            if results.multi_hand_landmarks:
                for handLms, handInfo in zip(results.multi_hand_landmarks, results.multi_handedness):
                    handType = handInfo.classification[0].label  # Get hand type (left or right)
                    mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
                    fingers_open = control_fan(handLms, handType)

                    # Swap left and right labels
                    display_hand_type = 'Right' if handType == 'Left' else 'Left'

                    cv2.putText(img, f'{display_hand_type} Hand, Fingers: {fingers_open}', (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

            cv2.imshow("Image", img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()
        ser.close()

if __name__ == "__main__":
    main()



KeyboardInterrupt: 